<a href="https://colab.research.google.com/github/bujardeari/lightweight-finetuning-off-foundation-model/blob/main/peft_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from peft import PeftModel, PeftConfig


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = load_dataset("imdb")

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model.config.pad_token_id = model.config.eos_token_id
model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

pre_trained_eval_results = trainer.evaluate()
print("Pre-trained model evaluation results:", pre_trained_eval_results)

# Define LoRA Config
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj", "c_fc"]
)

# Create PEFT model
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

# Set up the trainer for fine-tuning
peft_training_args = TrainingArguments(
    output_dir="./peft_results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./peft_logs',
    logging_steps=10,
    eval_strategy="epoch",
    fp16=True,
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

peft_trainer.train()

peft_model.save_pretrained("./peft_model")

fine_tuned_model = PeftModel.from_pretrained(model, "./peft_model")

eval_trainer = Trainer(
    model=fine_tuned_model,
    args=TrainingArguments(output_dir="./pre_trained_eval_results"),
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

fine_tuned_model_eval_result = eval_trainer.evaluate()
print("Fine-tuned model evaluation results:", fine_tuned_model_eval_result)

# Compare results
print("Pre-trained model accuracy:", pre_trained_eval_results['eval_accuracy'])
print("Fine-tuned model accuracy:", fine_tuned_model_eval_result['eval_accuracy'])
print("Improvement:", fine_tuned_model_eval_result['eval_accuracy'] - pre_trained_eval_results['eval_accuracy'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

Pre-trained model evaluation results: {'eval_loss': 0.7682700157165527, 'eval_model_preparation_time': 0.0026, 'eval_accuracy': 0.48764, 'eval_f1': 0.6379899951954329, 'eval_precision': 0.49324826290259144, 'eval_recall': 0.90296, 'eval_runtime': 308.1218, 'eval_samples_per_second': 81.137, 'eval_steps_per_second': 2.538}
trainable params: 2,360,832 || all params: 126,802,176 || trainable%: 1.8618


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.283800,0.205981,0.921320,0.921687,0.917413,0.926000


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Fine-tuned model evaluation results: {'eval_loss': 0.20598621666431427, 'eval_model_preparation_time': 0.0075, 'eval_accuracy': 0.92136, 'eval_f1': 0.9217294370570907, 'eval_precision': 0.9174195593596449, 'eval_recall': 0.92608, 'eval_runtime': 422.9622, 'eval_samples_per_second': 59.107, 'eval_steps_per_second': 7.388}
Pre-trained model accuracy: 0.48764
Fine-tuned model accuracy: 0.92136
Improvement: 0.43371999999999994
